# Uma análise quantitativa dos hotéis em Recife a partir dos dados do Trivago

Ângelo de Sant'Ana &mdash;
assd {em} cin.ufpe.br;

Antônio Rodrigues &mdash;
acpr {em} cin.ufpe.br;

Juliana Carvalho &mdash;
jco2 {em} cin.ufpe.br

junho de 2017

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import re

##### Localização dos dados relevantes

Abaixo começamos a localização dos elementos importantes das páginas .html para reconhecimento dos conjuntos que contêm
os valores que desejamos usar na nossa análise.

###### Dados sobre avaliação dos hotéis

In [18]:
# fazer um iterador para capturar todas as páginas de 1 a 5

i = 1
while i <= 5:
    
    urlAvaliacoes = "pages/avaliacoesPag%d.html" % i

    parsed_page_avaliacoes = BeautifulSoup(open(urlAvaliacoes,encoding="utf-8"),"html.parser")

    listaHoteis = []

    nomesDosHoteisAvaliacoes = parsed_page_avaliacoes.find_all('h3', class_="name__copytext m-0 item__slideout-toggle")

    divsAvaliacoes = parsed_page_avaliacoes.find_all('div', class_="rat-chart__group rat-chart__group--collapsed")

    avaliacaoToInt = lambda x: int(re.sub('[\/][0-9]{3}', '', re.sub('[a-zA-ZzáàâãéèêíïóôõöúçñÁÀÂÃÉÈÍÏÓÔÕÖÚÇÑ\s]+','',x)))

    aggregate = list(zip(nomesDosHoteisAvaliacoes, divsAvaliacoes))

    for (hotelName, avaliacoes) in aggregate:
        valoresAvaliacoes = avaliacoes.find_all(class_="rat-chart__number");

        listaValoresAux = []
        listaValoresAux.append(hotelName.text)

        for valor in valoresAvaliacoes:
            if "Não" in valor.text:
                listaValoresAux.append(valor.text)
            else:
                listaValoresAux.append(int(avaliacaoToInt(valor.text)))

    #         print(listaValoresAux)

        listaHoteis.append(dict(zip(["nome_do_hotel", "localizacao", "quartos", "servico", "limpeza", "custo-beneficio", \
                                     "conforto", "facilidades", "predio", "cafe_da_manha", "comida"], listaValoresAux)))

    # print(listaHoteis)

    pdHoteis = pd.DataFrame(listaHoteis, columns=["nome_do_hotel", "localizacao", "quartos", "servico", "limpeza", \
                                          "custo-beneficio", "conforto", "facilidades", "predio", "cafe_da_manha", "comida"])
    # print(pdHoteis.describe())
    # pdHoteis.head(70)
    arquivoSaida = 'saidaDataScraping/saidaDataScrapingAval%d.csv' % i
    pdHoteis.to_csv(arquivoSaida,sep=';',encoding='utf-8')
    i += 1


##### Dados sobre informações dos hotéis

In [16]:
urlInfos = "pages/InfoPag1.html"

parsed_page_infos = BeautifulSoup(open(urlInfos,encoding="utf-8"),"html.parser")

listaHoteis = []

# pegando os nomes do hotéis
nomesDosHoteisInfos = parsed_page_infos.find_all('h3', class_="name__copytext m-0 item__slideout-toggle")

# pegando o conjunto de comodidades de cada hotel
divsInfos = parsed_page_infos.find_all('div', class_="sl-box__block clearfix")

# associando o hotel as suas comodidades
aggregate = list(zip(nomesDosHoteisInfos, divsInfos))

# print(divsInfos)

# print(aggregate)

dictHoteisInfos = {}
pdInfos = pd.DataFrame()

listaAuxMerge = []

for (hotelName, hidden) in aggregate:    
    listaColumnsAux = []
    
    for infos in hidden:
        dictCaracteristicasAux = {}
        
        definiuDF = False
        
        if "Comodidades do Quarto" not in infos.text:
            for itens in infos.contents[1].find_all("li", class_="unordered-list__item"):
                listaColumnsAux.append(itens.contents[0])
                dictCaracteristicasAux[itens.contents[0]] = [1]
                
                if not definiuDF:
                    pdTemp = pd.DataFrame(dictCaracteristicasAux, columns=listaColumnsAux, index=[0])
                    definiuDF = True
#                 else:
#                     pdTemp[0] = pdTemp[itens.contents[0]].map({itens.contents[0]: 1})
        
            dictHoteisInfos[hotelName.text] = dictCaracteristicasAux
        
#         print(dictCaracteristicasAux)
            
    listaAuxMerge = list(set(listaAuxMerge) | set(listaColumnsAux))

pdTemp.head()
# print(dictHoteisInfos)
# print(listaAuxMerge)
# print(pdInfos)


,Academia,Bar no hotel,Check-in / out Expresso,Estacionamento,Lavanderia,PC com Internet,Piscina exterior,Quarto para não-fumantes,Recepção 24 horas,Restaurante,Sala de conferências,Terraço,Wi-Fi grátis no lobby,Wi-Fi no Lobby,Acessível para cadeirante,Corredores com acessibilidade,Cama infantil / Berço
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
